In [1]:
import torch
import torchaudio
import bigvgan
import librosa
import soundfile as sf
from vocos import Vocos
import speechbrain as sb
from speechbrain.utils.fetching import fetch
from speechbrain.utils.data_utils import split_path
from speechbrain.lobes.models.FastSpeech2 import mel_spectogram
from speechbrain.inference.vocoders import HIFIGAN
from speechbrain.inference.vocoders import DiffWaveVocoder
from transformers import UnivNetFeatureExtractor, UnivNetModel
import librosa
import librosa.display
import numpy as np
from pathlib import Path
from scipy.io.wavfile import write
import IPython.display as ipd
from tqdm import tqdm
import os
import time
from torch import nn

c:\Studia\.conda\Lib\inspect.py:988: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):


In [3]:
src = [str(file) for file in Path('noised_data').rglob('*') if file.is_file()]

In [7]:
for src_path in tqdm(src):
    new_path = 'noised_data_preprocessed' + src_path[11:]
    src_waveform, src_sr = torchaudio.load(src_path)
    src_waveform = librosa.resample(src_waveform.numpy(), orig_sr=src_sr, target_sr=16000)
    torchaudio.save(new_path, torch.Tensor(src_waveform), 16000)

100%|██████████| 1439/1439 [00:11<00:00, 127.50it/s]


In [17]:
src = [str(file) for file in Path('noised_data_preprocessed').rglob('*') if file.is_file()]

In [18]:
for model in ['bigvgan', 'diffwave', 'hifigan', 'univnet', 'vocos', 'waveglow', 'griffinlim'][-1:]:
    gen = [str(file) for file in Path(f'noised_generated/{model}').rglob('*') if file.is_file()]
    for src_path, gen_path in tqdm(list(zip(src, gen))):
        new_path = 'noised_generated_preprocessed' + gen_path[16:]
        src_waveform, src_sr = torchaudio.load(src_path)
        gen_waveform, gen_sr = torchaudio.load(gen_path)

        src_waveform = src_waveform[0].numpy()
        gen_waveform = gen_waveform[0].numpy()

        # zmiana próbkowania dla dopasowania do oryginału
        if src_sr != gen_sr:
            new_gen_wave = librosa.resample(gen_waveform, orig_sr=gen_sr, target_sr=src_sr)
        else:
            new_gen_wave = gen_waveform
            
        # new_gen_wave = librosa.effects.time_stretch(new_gen_wave, rate=(len(new_gen_wave) / len(src_waveform))) # dopasowanie długości
        new_gen_wave *= float(max(np.abs(src_waveform)) / max(np.abs(new_gen_wave))) # dostosowanie amplitudy

        torchaudio.save(new_path, torch.Tensor(new_gen_wave).unsqueeze(0), src_sr)

100%|██████████| 1439/1439 [00:27<00:00, 52.18it/s]
